In [ ]:
# data to remove
# data_df[data_df['selftext'].str.contains('\[removed\]')]

In [1]:
import time
import datetime as dt

import json

import configparser

import requests
from pmaw import PushshiftAPI

import pandas as pd

In [2]:
headers = {'User-Agent': 'test_project'}

# Pulling posts using pmaw
* this method actually works in filtering using before and after
* multithreaded library allows for faster pulling of data

[Source](https://www.reddit.com/r/pushshift/comments/y4raqy/keyerror_created_utc_in_pushshiftapibasepy/)

In [3]:
def fxn(item):
    return item['link_id'] in post_ids

In [4]:
def get_date_range(before, after):
    before_str = f"{dt.datetime.fromtimestamp(before).month}_{dt.datetime.fromtimestamp(before).year}"
    after_str = f"{dt.datetime.fromtimestamp(after).month}_{dt.datetime.fromtimestamp(after).year}"
    return f"{after_str}_to_{before_str}"

In [5]:
def write_posts(post_list, before, after):
    keys_interest = ['subreddit', 'title', 'id', 'author', 'created_utc', 'selftext', 'full_link']
    data_dict = {}
    res_dict = {}
    for i, post_dict in enumerate(post_list):
        res_dict[i] = post_dict
        data_dict[i] = {}
        for k in keys_interest:
            try:
                data_dict[i][k] = post_dict[k]
            except:
                data_dict[i][k] = None

    # Write json files of original pull
    date_str = get_date_range(before, after)
    with open(f'sd_post_{date_str}.json', 'w') as f:
        json.dump(res_dict, f)

    # Write out csv of data
    data_df = pd.DataFrame(data_dict).T
    try:
        data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
        data_df.sort_values('created_utc')
    except:
        print(f"Failed to convert timestamps for: sd_post_data_{date_str}")
    data_df.reset_index(inplace=True, drop=True)
    data_df.to_csv(f'sd_post_{date_str}.csv', index=False)
    post_ids = 't3_' + data_df.id.values
    post_ids = post_ids.tolist()
    return post_ids

def write_comments(comment_list, before, after):
    keys_interest = ['subreddit', 'link_id', 'author', 'body', 'score', 'id', 'created_utc', 'permalink']
    data_dict = {}
    res_dict = {}
    for i, comm_dict in enumerate(comment_list):
        res_dict[i] = comm_dict
        data_dict[i] = {}
        for k in keys_interest:
            try:
                if k == 'permalink':
                    comment = comm_dict[k]
                    title = comment.split('/')[-3]
                    data_dict[i]['title'] = title
                elif k == 'link_id':
                    post_id = comm_dict[k].split('_')[-1]
                    data_dict[i]['post_id'] = post_id
                data_dict[i][k] = comm_dict[k]
            except:
                data_dict[i][k] = None

    # Write json files of original pull
    date_str = get_date_range(before, after)
    with open(f'sd_comment_{date_str}.json', 'w') as f:
        json.dump(res_dict, f)

    # Write out csv of data
    comm_data_df = pd.DataFrame(data_dict).T
    comm_data_df["created_utc"] = pd.to_datetime(comm_data_df["created_utc"], utc=True, unit='s')
    comm_data_df.reset_index(inplace=True, drop=True)
    comm_data_df.sort_values('created_utc')
    comm_data_df.drop_duplicates(inplace=True)
    comm_data_df.to_csv(f'sd_comment_{date_str}.csv', index=False)

In [6]:
# before = int(dt.datetime(2022, 11, 1, 0, 0).timestamp())
# after = int(dt.datetime(2021, 12, 31, 23, 59).timestamp())
all_dates = [
            # (int(dt.datetime(2021, 12, 31, 23, 59).timestamp()), int(dt.datetime(2022, 1, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 1, 31, 0, 0).timestamp()), int(dt.datetime(2022, 2, 28, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 2, 28, 0, 0).timestamp()), int(dt.datetime(2022, 3, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 3, 31, 0, 0).timestamp()), int(dt.datetime(2022, 4, 30, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 4, 30, 0, 0).timestamp()), int(dt.datetime(2022, 5, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 5, 31, 0, 0).timestamp()), int(dt.datetime(2022, 6, 30, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 6, 30, 0, 0).timestamp()), int(dt.datetime(2022, 7, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 7, 31, 0, 0).timestamp()), int(dt.datetime(2022, 8, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 8, 31, 0, 0).timestamp()), int(dt.datetime(2022, 9, 30, 0, 0).timestamp())),
            (int(dt.datetime(2022, 9, 30, 0, 0).timestamp()), int(dt.datetime(2022, 10, 31, 0, 0).timestamp())),
            # (int(dt.datetime(2022, 10, 31, 0, 0).timestamp()), int(dt.datetime(2022, 11, 15, 0, 0).timestamp()))
            ]
# for after, before in all_dates:
#     print(get_date_range(before, after))

In [7]:
# Code pulled from reddit post link below
# https://www.reddit.com/r/pushshift/comments/y4raqy/keyerror_created_utc_in_pushshiftapibasepy/

subreddits = ['sandiego', 'sandiegan', 'ucsd', 'usd', 'sdsu']

submissions_limit = 100
comments_limit = 20
# users_limit = 350

api = PushshiftAPI()

# before = int(dt.datetime(2022,11,1,0,0).timestamp())
# after = int(dt.datetime(2022,10,1,0,0).timestamp())
# before = int(dt.datetime(2022, 11, 1, 0, 0).timestamp())
# after = int(dt.datetime(2021, 12, 31, 23, 59).timestamp())
for after, before in all_dates:
    start_time = time.time()
    collecting_users_submissions = api.search_submissions(
                                        subreddit=subreddits,
                                        # limit=comments_limit,
                                        before=before, after=after,
                                    )
    # print(f'Retrieved {len(collecting_users_submissions)} posts')
    print(f'Elapsed time: {time.time() - start_time}')
    post_list = [post for post in collecting_users_submissions]
    post_ids = write_posts(post_list, before, after)
    print(f"Total Posts: {len(post_list)}")

    start_time = time.time()
    try:
        collecting_users_comments = api.search_comments(
                                            subreddit=subreddits,
                                            # limit=comments_limit,
                                            before=before, after=after,
                                            sort_type='score', sort='desc',
                                            size=3,
                                            # max_results_per_request=10,
                                            filter_fn=fxn,
                                            # fields=['author', 'subreddit', 'created_utc']
                                        )
        comment_list = [comment for comment in collecting_users_comments]
        print(f'Elapsed time: {time.time() - start_time}')
        write_comments(comment_list, before, after)
        print(f"Total Comments: {len(comment_list)}")
    except:
        print(f"Failed to get comments for: {get_date_range(before, after)}")

Elapsed time: 148.00704908370972
Total Posts: 5002
Elapsed time: 223.91387391090393
Total Comments: 7413


In [4]:
keys_interest = ['subreddit', 'title', 'id', 'author', 'created_utc', 'selftext', 'full_link']
data_dict = {}
res_dict = {}
for i, post_dict in enumerate(post_list):
    res_dict[i] = post_dict
    data_dict[i] = {}
    for k in keys_interest:
        try:
            data_dict[i][k] = post_dict[k]
        except:
            data_dict[i][k] = None

# Write json files of original pull
with open('sd_post_data.json', 'w') as f:
    json.dump(res_dict, f)
    
# Write out csv of data
data_df = pd.DataFrame(data_dict).T
data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
data_df.reset_index(inplace=True, drop=True)
data_df.sort_values('created_utc')
data_df.to_csv('sd_post_data.csv', index=False)
data_df

,subreddit,title,id,author,created_utc,selftext,full_link
0,sandiego,How to find a room under 1100??,t5hc56,logancali,2022-03-03 02:53:22+00:00,I am a San Diego native and have been looking ...,https://www.reddit.com/r/sandiego/comments/t5h...
1,UCSD,need tips and tricks to ace ur finals? i gotchu 😎,t5hb3u,_GVTS_,2022-03-03 02:51:56+00:00,,https://www.reddit.com/r/UCSD/comments/t5hb3u/...
2,sandiego,Tax Return Intercepted by City?,t5h7cp,abn104,2022-03-03 02:46:25+00:00,My FTB account says that $250+ of my Californi...,https://www.reddit.com/r/sandiego/comments/t5h...
3,sandiego,San Diego Convention Center,t5h2k4,4gecko44,2022-03-03 02:39:21+00:00,&amp;#x200B;\n\n[At the direction of San Diego...,https://www.reddit.com/r/sandiego/comments/t5h...
4,UCSD,Grubhub and Starbucks?,t5gyj4,electric_syrup_619,2022-03-03 02:33:42+00:00,Does anyone know how to order at UCSD's Starbu...,https://www.reddit.com/r/UCSD/comments/t5gyj4/...
...,...,...,...,...,...,...,...
49873,UCSD,Does CHEM 7L consist of individual or group labs?,tqechl,MinuteStuff5450,2022-03-28 17:20:00+00:00,have my first lab today and just want to menta...,https://www.reddit.com/r/UCSD/comments/tqechl/...
49874,sandiego,"50 Years Ago Today (March 28, 1971) A group of...",tqe9ya,PlugInDaPhone,2022-03-28 17:16:45+00:00,,https://www.reddit.com/r/sandiego/comments/tqe...
49875,sandiego,I don't give 2 F#$ks about WS slapping CR,tqe6qo,TarnMaster1985,2022-03-28 17:12:53+00:00,"And neither should you. In fact, I am annoyed...",https://www.reddit.com/r/sandiego/comments/tqe...
49876,UCSD,Anyone know any open upper psych classes?,tqdyb0,jaiajwbrofsjapnd,2022-03-28 17:02:41+00:00,"I’m supposed to graduate this quarter, however...",https://www.reddit.com/r/UCSD/comments/tqdyb0/...


In [13]:
# post_ids = data_df.id.values.tolist()
post_ids = 't3_' + data_df.id.values
post_ids = post_ids.tolist()
len(post_ids)
# post_ids = 't3_' _data_df.id

49878

In [19]:
start_time = time.time()
api = PushshiftAPI()
def fxn(item):
    return item['link_id'] in post_ids
# posts = api.search_submissions(ids=post_ids, filter_fn=fxn)

collecting_users_comments = api.search_comments(
                                    subreddit=subreddits,
                                    # limit=comments_limit,
                                    before=before, after=after,
                                    sort_type='score', sort='desc',
                                    size=3,
                                    # max_results_per_request=10,
                                    filter_fn=fxn,
                                    # fields=['author', 'subreddit', 'created_utc']
                                )
print(f'Elapsed time: {time.time() - start_time}')
comment_list = [comment for comment in collecting_users_comments]
print(f'Elapsed time: {time.time() - start_time}')

Elapsed time: 879.6556451320648
Elapsed time: 879.7017271518707


In [20]:
keys_interest = ['subreddit', 'link_id', 'author', 'body', 'score', 'id', 'created_utc', 'permalink']
data_dict = {}
res_dict = {}
for i, comm_dict in enumerate(comment_list):
    res_dict[i] = comm_dict
    data_dict[i] = {}
    for k in keys_interest:
        try:
            if k == 'permalink':
                comment = comm_dict[k]
                title = comment.split('/')[-3]
                data_dict[i]['title'] = title
            elif k == 'link_id':
                post_id = comm_dict[k].split('_')[-1]
                data_dict[i]['post_id'] = post_id
            data_dict[i][k] = comm_dict[k]
        except:
            data_dict[i][k] = None

# Write json files of original pull
with open('test_sd_comment_data.json', 'w') as f:
    json.dump(res_dict, f)
    
# Write out csv of data
comm_data_df = pd.DataFrame(data_dict).T
comm_data_df["created_utc"] = pd.to_datetime(comm_data_df["created_utc"], utc=True, unit='s')
comm_data_df.reset_index(inplace=True, drop=True)
comm_data_df.sort_values('created_utc')
comm_data_df.drop_duplicates(inplace=True)
comm_data_df.to_csv('test_sd_comment_data.csv', index=False)

,subreddit,post_id,link_id,author,body,score,id,created_utc,title,permalink
0,sandiego,sbzzrr,t3_sbzzrr,goingplaidinum,It's not even the best bar crawl spot in San D...,25,hu387uy,2022-01-25 00:38:21+00:00,gaslamp_is_the_best_bar_crawl_place_in_america,/r/sandiego/comments/sbzzrr/gaslamp_is_the_bes...
1,sandiego,sc7tsy,t3_sc7tsy,MagyarCat,Wtf is this,18,hu4klfs,2022-01-25 07:28:12+00:00,came_home_tonight_to_see_a_love_white_people,/r/sandiego/comments/sc7tsy/came_home_tonight_...
2,UCSD,sc6jsy,t3_sc6jsy,Paging_JimmyPage,Oh hey just wanted to say thanks for lending m...,17,hu4jx4d,2022-01-25 07:20:13+00:00,soo_what_are_yall_doing_for_valentines_day,/r/UCSD/comments/sc6jsy/soo_what_are_yall_doin...
3,sandiego,sbxseo,t3_sbxseo,SanDiego_Account,"""My religion forbids me from getting a vaccine...",15,hu39wcj,2022-01-25 00:52:41+00:00,san_diego_unified_teachers_given_termination,/r/sandiego/comments/sbxseo/san_diego_unified_...
4,sandiego,s44es9,t3_s44es9,quantum_altar,actually very clean and tidy compared to alot ...,15,hsp45yq,2022-01-14 23:45:41+00:00,my_work_dumpsters_as_the_garbage_strike_rolls_on,/r/sandiego/comments/s44es9/my_work_dumpsters_...
...,...,...,...,...,...,...,...,...,...,...
32567,sandiego,wekanp,t3_wekanp,Terminator_Conn,[USPS Free Covid tests](https://special.usps.c...,1,iiolqu9,2022-08-02 19:23:24+00:00,free_covid_test,/r/sandiego/comments/wekanp/free_covid_test/ii...
32568,sandiego,weg0qh,t3_weg0qh,psnanda,Oh dang . Goes to show that $100k has become t...,1,iioly3k,2022-08-02 19:24:41+00:00,i_make_23_an_hour_and_am_still_having_issues,/r/sandiego/comments/weg0qh/i_make_23_an_hour_...
32569,UCSD,vws8cc,t3_vws8cc,ravens_house,"especially for your first quarter, take it slo...",1,iiolyp0,2022-08-02 19:24:47+00:00,incoming_student_enrollment_megathread,/r/UCSD/comments/vws8cc/incoming_student_enrol...
32570,sandiego,weg0qh,t3_weg0qh,0nlythebest,this is a great answer. Im debating if I want ...,1,iiom5uu,2022-08-02 19:26:00+00:00,i_make_23_an_hour_and_am_still_having_issues,/r/sandiego/comments/weg0qh/i_make_23_an_hour_...


In [21]:
before

1667286000

In [24]:
f"{dt.datetime.fromtimestamp(before).month}_{dt.datetime.fromtimestamp(before).year}"

'11_2022'

In [25]:
f"{dt.datetime.fromtimestamp(after).month}_{dt.datetime.fromtimestamp(after).year}"

'12_2021'

In [29]:
get_date_range(before, after)

'11_2022_to_12_2021'

In [7]:
len(comm_data_df.post_id.unique())

4609

In [17]:
comment_list[0]['link_id'] in post_ids

True

In [7]:
# comment_ids = api.search_submission_comment_ids(ids=data_df.id)
# comment_id_list = [c_id for c_id in comment_ids]

In [8]:
comment_ids

In [12]:
start_time = time.time()
api = PushshiftAPI()
def fxn(item):
    return item['score'] > 5
# posts = api.search_submissions(ids=post_ids, filter_fn=fxn)

collecting_users_comments = api.search_comments(
                                    subreddit=subreddits,
                                    # limit=comments_limit,
                                    before=before, after=after,
                                    max_results_per_request=10,
                                    filter_fn=fxn,
                                    # fields=['author', 'subreddit', 'created_utc']
                                )
print(f'Elapsed time: {time.time() - start_time}')
comment_list = [comment for comment in collecting_users_comments]
print(f'Elapsed time: {time.time() - start_time}')

Elapsed time: 1021.0304386615753
Elapsed time: 1021.031539440155


In [50]:
comm_data_df[comm_data_df.title.str.contains('gi_bil')]

,subreddit,id,author,body,score,created_utc,title,permalink
142,SDSU,iunc3nt,kellyoceanmarine,It will not cover out of state tuition. Trust ...,1,2022-11-01 17:11:20+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
249,SDSU,iuopbt2,SCOveterandretired,"If you move there, you should be charged in-st...",1,2022-11-01 22:24:53+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
253,SDSU,iuoqvbz,GovernmentNo8474,You have to be accepted for it each semester w...,1,2022-11-01 22:35:34+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
1248,SDSU,iuopbt2,SCOveterandretired,"If you move there, you should be charged in-st...",1,2022-11-01 22:24:53+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
1252,SDSU,iuoqvbz,GovernmentNo8474,You have to be accepted for it each semester w...,1,2022-11-01 22:35:34+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
1654,SDSU,iunc3nt,kellyoceanmarine,It will not cover out of state tuition. Trust ...,1,2022-11-01 17:11:20+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...
2936,SDSU,iunc3nt,kellyoceanmarine,It will not cover out of state tuition. Trust ...,1,2022-11-01 17:11:20+00:00,anybody_here_using_gi_bill_that_can_answer_my,/r/SDSU/comments/yjdkrz/anybody_here_using_gi_...


In [58]:
comm_data_df.to_csv('./test_comm_df.csv')
# Write json files of original pull
with open('test_comm_sd_data.json', 'w') as f:
    json.dump(res_dict, f)

In [24]:
test_comm = comm_data_df.permalink[0]
test_comm.split('/')[-3]

'me_my_husband_cat_are_going_to_be_homeless_in'

In [26]:
len(comm_data_df.title.unique())

473

In [19]:
comm_data_df.score.max()

362

In [56]:
keys_interest = ['subreddit', 'title', 'id', 'author', 'created_utc', 'selftext', 'full_link']
data_dict = {}
res_dict = {}
for i, post_dict in enumerate(post_list):
    res_dict[i] = post_dict
    data_dict[i] = {}
    for k in keys_interest:
        try:
            data_dict[i][k] = post_dict[k]
        except:
            data_dict[i][k] = None

# Write json files of original pull
with open('sd_data.json', 'w') as f:
    json.dump(res_dict, f)
    
# Write out csv of data
data_df = pd.DataFrame(data_dict).T
data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
data_df.reset_index(inplace=True, drop=True)
data_df.sort_values('created_utc')
data_df.to_csv('sd_data.csv', index=False)
data_df

,subreddit,title,id,author,created_utc,selftext,full_link
0,UCSD,WHY is the calc midterm at 8pm,yj9dof,lastminutemaniac,2022-11-01 13:56:50+00:00,I bought vip tickets to a concert bout a year ...,https://www.reddit.com/r/UCSD/comments/yj9dof/...
1,sandiego,Spooky Halloween Sunset / San Marcos,yj98mk,tomf_182,2022-11-01 13:50:48+00:00,,https://www.reddit.com/r/sandiego/comments/yj9...
2,sandiego,"My run was amazing today, thanks San Diego! So...",yj3hkd,KryptonicHD,2022-11-01 08:59:04+00:00,,https://www.reddit.com/r/sandiego/comments/yj3...
3,sandiego,"San Diego sunsets are crazy, but today's was m...",yj3dyq,KryptonicHD,2022-11-01 08:53:54+00:00,,https://www.reddit.com/r/sandiego/comments/yj3...
4,sandiego,Visiting for the first time!,yj2ypc,scoobieAdoobie,2022-11-01 08:29:38+00:00,Coming to town for a couple days in 2 weeks an...,https://www.reddit.com/r/sandiego/comments/yj2...
...,...,...,...,...,...,...,...
553,SanDiegan,All Southbound I-15 Traffic Being Detoured to ...,ylj3ej,Lamacorn,2022-11-03 23:57:12+00:00,,https://www.reddit.com/r/SanDiegan/comments/yl...
554,UCSD,Psyc 106 - First/Second Pass?,ylixs4,AkihikoRyuu,2022-11-03 23:50:16+00:00,Anyone know how fast Psyc 106 fills up? Just t...,https://www.reddit.com/r/UCSD/comments/ylixs4/...
555,sandiego,Worlds Largest Neuroscience Conference Returns...,ykdrvi,SD_TMI,2022-11-02 19:02:04+00:00,,https://www.reddit.com/r/sandiego/comments/ykd...
556,UCSD,sio 20?,ykdr34,Sharp-Farmer1158,2022-11-02 19:01:18+00:00,Anyone reccomened sio 20? Idk the professor bu...,https://www.reddit.com/r/UCSD/comments/ykdr34/...


In [ ]:

# 70	FEED ME SEYMOUR
# feed_me_seymour

# Other Methods of Interfacing with reddit posts

## WARNING: No Guarantee These Methods or Code Works
All code was used as exploration in getting information from reddit

# Pulling posts using Reddit's api

## Issues
* No way to pull posts from specific times

In [2]:
subreddit      = "SanDiego"
subreddit_filt = "new"

In [3]:
# read in config file
config = configparser.ConfigParser()
config.read('reddit_config.ini')

['reddit_config.ini']

In [4]:
# Request access to reddit
auth = requests.auth.HTTPBasicAuth(config["script"]["personal_use_script"], config["script"]["secret"])

# set up log in creds
data = {'grant_type': 'password',
        'username': config["auth"]["username"],
        'password': config["auth"]["password"]
       }

In [5]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [6]:
res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{subreddit_filt}",
                   headers=headers, params={'limit':'200'})

In [7]:
reddit_res = res.json()

In [8]:
keys_interest = ['title', 'created_utc', 'selftext', 'url', 'link_flair_text']
data_dict = {}
for i, d in enumerate(reddit_res['data']['children']):
    data_dict[i] = {k: d['data'][k] for k in keys_interest}
data_df = pd.DataFrame(data_dict).T
data_df

,title,created_utc,selftext,url,link_flair_text
0,San Diego Zoo!,1667595536.0,I apologize I assume you guys get a lot of the...,https://www.reddit.com/r/sandiego/comments/yma...,None
1,Mexico Soccer Jersey,1667593178.0,where can i found some in sd?,https://www.reddit.com/r/sandiego/comments/ym9...,None
2,Stratotanker Currently Circling San Diego,1667592914.0,Anyone else see/hear the Stratotanker currentl...,https://www.reddit.com/r/sandiego/comments/ym9...,None
3,Point Loma Tide Pools,1667592754.0,,https://i.redd.it/0nyaaektrzx91.jpg,Photo
4,Car accident on intersection Texas St and Cami...,1667591281.0,,https://i.redd.it/snnbsnrhnzx91.jpg,Photo
...,...,...,...,...,...
95,How to join pickleball team in SD?,1667369435.0,Any pickleball players have insight on how to ...,https://www.reddit.com/r/sandiego/comments/yjx...,None
96,Need help from you fine folks again…,1667368811.0,"Except this time instead of “open late,” are t...",https://www.reddit.com/r/sandiego/comments/yjx...,None
97,Tequila samples,1667367930.0,Does anyone know where I can find a liquor sto...,https://www.reddit.com/r/sandiego/comments/yjw...,None
98,My 16yo’s car was stolen near SDSU. Cops don’t...,1667366199.0,,https://www.reddit.com/r/sandiego/comments/yjw...,None


In [9]:
data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
data_df

,title,created_utc,selftext,url,link_flair_text
0,San Diego Zoo!,2022-11-04 20:58:56+00:00,I apologize I assume you guys get a lot of the...,https://www.reddit.com/r/sandiego/comments/yma...,None
1,Mexico Soccer Jersey,2022-11-04 20:19:38+00:00,where can i found some in sd?,https://www.reddit.com/r/sandiego/comments/ym9...,None
2,Stratotanker Currently Circling San Diego,2022-11-04 20:15:14+00:00,Anyone else see/hear the Stratotanker currentl...,https://www.reddit.com/r/sandiego/comments/ym9...,None
3,Point Loma Tide Pools,2022-11-04 20:12:34+00:00,,https://i.redd.it/0nyaaektrzx91.jpg,Photo
4,Car accident on intersection Texas St and Cami...,2022-11-04 19:48:01+00:00,,https://i.redd.it/snnbsnrhnzx91.jpg,Photo
...,...,...,...,...,...
95,How to join pickleball team in SD?,2022-11-02 06:10:35+00:00,Any pickleball players have insight on how to ...,https://www.reddit.com/r/sandiego/comments/yjx...,None
96,Need help from you fine folks again…,2022-11-02 06:00:11+00:00,"Except this time instead of “open late,” are t...",https://www.reddit.com/r/sandiego/comments/yjx...,None
97,Tequila samples,2022-11-02 05:45:30+00:00,Does anyone know where I can find a liquor sto...,https://www.reddit.com/r/sandiego/comments/yjw...,None
98,My 16yo’s car was stolen near SDSU. Cops don’t...,2022-11-02 05:16:39+00:00,,https://www.reddit.com/r/sandiego/comments/yjw...,None


In [10]:
reddit_res['data']['children'][0]['data'].keys()

dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'post_hint', 'content_categories', 'is_self', 'subreddit_type', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html',

In [11]:
reddit_res['data'].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])

In [12]:
data_df.to_csv("reddit_cop_example.csv", index=False)

In [13]:
with open('reddit_cop_data.json', 'w') as f:
    json.dump(reddit_res['data'], f)

# Pulling using Push Shift

## Issues
* before and after filter doesn't seem to work to filter posts
    * [Click here for example](https://api.pushshift.io/reddit/submission/search?limit=1000&?after=1667276000&before=1667286000&sort_type=score&sort=desc&subreddit=SanDiego)

In [9]:
start = dt.datetime(2022, 11, 1, 0, 0)
before = int(start.timestamp())
after = before - 10000

# after = datetime(2021, 12, 31, 23, 59)
# after = int(start.timestamp())

end = dt.datetime(2022, 9, 1, 23, 59) # testing purposes
final_after = int(end.timestamp())

print((before, after))

(1667286000, 1667276000)


True

In [10]:
# Putting together and testing various urls

# url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&before={}&after={}"
# url = "https://api.pushshift.io/reddit/submission/search/?after={}&before={}&sort_type=score&sort=desc&subreddit={}"
url = "https://api.pushshift.io/reddit/submission/search?limit=1000&?after={}&before={}&sort_type=score&sort=desc&subreddit={}"
# url = "https://api.pushshift.io/reddit/submission/search?limit=25&?before={}&sort_type=score&sort=desc&subreddit={}"

# url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&before={}&after={}"

In [11]:
# Get out example url
url.format(after, before, subreddit)

'https://api.pushshift.io/reddit/submission/search?limit=1000&?after=1667276000&before=1667286000&sort_type=score&sort=desc&subreddit=SanDiego'

In [13]:
res_df = pd.DataFrame()
i = 0
print(final_after < before)
while final_after < before:
    time.sleep(1)
    # new_url = url.format(subreddit, before, after)
    new_url = url.format(after, before, subreddit)
    print(new_url)
    # new_url = url.format(before, subreddit)
    res = requests.get(new_url, headers=headers)
    print(res)
    
    reddit_res = res.json()
    keys_interest = ['title', 'author', 'created_utc', 'selftext', 'full_link']
    data_dict = {}
    for i, d in enumerate(reddit_res['data']):
        data_dict[i] = {}
        for k in keys_interest:
            try:
                data_dict[i][k] = d[k]
            except:
                data_dict[i][k] = None
        # data_dict[i] = {k: d[k] for k in keys_interest}
    data_df = pd.DataFrame(data_dict).T
    # break
    data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
    res_df = pd.concat([res_df, data_df])
    res_df.reset_index(inplace=True)
    res_df.sort_values('created_utc')
    
    before = data_df.iloc[-1].created_utc.to_pydatetime()
    before = int(before.timestamp())
    after = before - 100000
    print(before)
    i = i + 1
    if i > 3:
        break
res_df.to_csv("test_output.csv", index=False)
res_df

True
1667286000
0
https://api.pushshift.io/reddit/submission/search?limit=1000&?after=1667276000&before=1667286000&sort_type=score&sort=desc&subreddit=SanDiego
<Response [200]>
1561748408
